In [4]:
import math
import numpy as np

Dictionary:

1. A "zonal flow" is one that follows latitudinal lines, and a "meridional flow" is one that follows longitudinal lines.

## Dimensionless linearized planetary geostrophic motion
Let $P_L$ be the pressure at some point in the atmosphere and $P^S_L$ the mean pressure $P_L^S=-Uy$ ($U$ described below).We wish to study the equation
$$\frac{\partial^2\phi_L}{\partial z^2}-\frac{1}{H}\frac{\partial\phi_L}{\partial z}+\frac{\beta_LS}{U}\phi_L=-\left(k_s+\frac{1}{H}\right)\frac{Q_0}{U}F(x)e^{-k_sz}.\qquad(7)$$
This models the relationship between

* $\phi_L$, a zonally asymmetric pressure perturbation such that $P_L=P_L^S+\phi_L$ and $\phi_L\ll P_L^S$.
* $U$, the dominant mean flow. This is a zonal mean barotropic wind driven by mean pressure $P_L^S=-Uy$.

We have a number of constants in the above equation:

* $\beta_L$ is the rate of change of the coriolis parameter with respect to meridional distance ($\beta_L=\frac{\partial f}{\partial x}$), where we parameterize $f$ linearly.
* $H$ is a constant scale height to account for how density changes with respect to height.
* $S$ is the dimensionless vertical stability (???).
* $k_s$ is a constant to treat thermal forcing adjacent to the surface.
* $Q_0$ is the amplitude of some zonally asymmetric thermal forcing.

### Discretization
Here we discretize (7):
$$\frac{\phi_L(z_{i+1})-2\phi_L(z_{i})+\phi_L(z_{i-1})}{(\Delta z)^2}-\frac{1}{H}\frac{\phi_L(z_{i+1})-\phi_L(z_{i})}{\Delta z}+\frac{\beta_LS}{U}\phi_L(z_i)=-\left(k_s+\frac{1}{H}\right)\frac{Q_0}{U}F(x)e^{-k_sz}.$$
Now we rewrite to solve for the next spacial step,
$$\phi_L(z_{i+1})=\frac{H}{H+\Delta z}\left(2\phi_L(z_i)-\phi_L(z_{i-1})-(\Delta z)^2\frac{\beta_LS}{U}\phi_L(z_i)-(\Delta z)^2\left(k_s+\frac{1}{H}\right)\frac{Q_0}{U}F(x)e^{-k_sz}\right)$$

### Boundary conditions
We consider boundary conditions determined by $w_L|_{z_0,z_T}=\left.\frac{\partial P_L}{\partial z}\right|_{z_0,z_T}=0$, where $z_0$ is the surface and $z_T$ is the maximum height under consideration. These conditions translate to the following statement about $\phi_L$ on the boundaries:

$$\frac{\partial}{\partial x}\frac{\partial\phi_L}{\partial z}=\frac{Q_0}{U}G(x)e^{-k_sz}$$

where $F(x)=G'(x)$. We make the simplifying assumption that $\phi_L=0$ on the boundaries.

In [11]:
# constants
beta = 1
H = 1
S = 1
k = 1
Q_0 = 1
U = 1

# stepping
nx, ny, nz = 10, 10, 10
dx, dy, dz = 1, 1, 1

# pre-allocate arrays
x = np.zeros(nx)
y = np.zeros(ny)
z = np.zeros(nz)
phi_L = np.zeros(nz)

# forcing function
F = lambda x: 1

# right-hand side of marching equation.
g = lambda i, phi_L: ( H / (H + dz) ) * ( 2*phi_L[i] - phi_L[i-1] - (dz**2) * ( ( beta*S / U) * phi_L[i] ) - (dz**2) * ( (k + (1 / H) ) * (Q_0 / U) * F(x[i]) * np.exp(-k*z[i]) ) )

def marchPhi_L():
    for i in range(0,nz):
        if i == 0 or i == nz-1:
            phi_L[i] = 0        # boundary condition
        if i != nz-1:
            phi_L[i+1] = g(i, phi_L)
        
    return phi_L

In [12]:
marchPhi_L()

array([ 0.       , -1.       , -1.5      , -1.25     , -0.875    ,
       -0.8125   , -0.96875  , -1.078125 , -1.0546875,  0.       ])

### Analytic solution
The authors provide analytic solutions to (7) under the (non-simplified) boundary conditions described above. The solutions depend on the sign of the discriminant of the characteristic polynomial. We provide both solutions below.

\begin{align*}
\text{Positive discriminant:}\quad\phi_L&=\frac{Q_0F(x)}{U(k_s^2+\frac{1}{H}k_s)+\beta_LS}\left(-\frac{e^{\frac{1}{2H}-q}-e^{-k_s}}{2e^{\frac{1}{2H}}\sinh(q)}e^{\left(\frac{1}{2H}+q\right)z}-\frac{e^{\frac{1}{2H}+q}-e^{-k_s}}{2e^{\frac{1}{2H}}\sinh(q)}e^{\left(\frac{1}{2H}-q\right)z}-\left(k_s+\frac{1}{H}\right)e^{-k_sz}\right)\\
&=A_0F(x)\left(B_0f_0(z)-C_0g_0(z)-D_0h_0(z)\right)
\end{align*}

\begin{align*}
\text{Negative discriminant:}\quad\phi_L&=\frac{Q_0F(x)}{U(k_s^2+\frac{1}{H}k_s)+\beta_LS}e^{\frac{z}{2H}}\left(\frac{\sin(mz-m)-e^{-\left(k_s+\frac{1}{2H}\right)}\sin(mz)}{\sin(m)}\right)+\frac{Q_0F(x)}{U(k_s^2+\frac{1}{H}k_s)+\beta_LS}e^{-k_sz}\\
&=A_0F(x)f_1(z)g_1(z)+A_0F(x)h_1(z)
\end{align*}

We have labeled the functions above for ease of readability in the code. In the above we have

$$q=\sqrt{\frac{1}{4H^2}-\frac{\beta_LS}{U}}.$$

In [18]:
q = np.sqrt( ( 1 / (4*(H**2)) ) - ( (beta*S) / (U) ) )

# positive discriminant
A_0 = Q_0 / (U*( k**2 + (1 / H)*k ) *beta*S)
B_0 = (np.exp((1/2*H)-q) - np.exp(-k)) / (2*np.exp(1/(2*H))*np.sinh(q))
C_0 = (np.exp((1/2*H)+q) - np.exp(-k)) / (2*np.exp(1/(2*H))*np.sinh(q))
D_0 = (k + (1/H))
f_0 = lambda z: np.exp( ( (1/(2*H)) + q )*z )
g_0 = lambda z: np.exp( ( (1/(2*H)) - q )*z )
h_0 = lambda z: np.exp(-k * z)

phi_Lpos = lambda z,x: A_0*F(x)( B_0*f_0(z) - C_0*g_0(z) - D_0*h_0(z) )

# negative discriminant
f_1 = lambda z: np.exp(z / (2*H))
g_1 = lambda z: ( np.sin(m*z - m) - np.exp(k_s+(1/(2*H)))*np.sin(m*z) ) / (np.sin(m))
h_1 = lambda z: np.exp(-k * z)

phi_Lneg = lambda z,x: A_0*F(x)*f_1(z)*g_1(z) + A_0*F(x)*h_1(z)

/tmp/ipykernel_3830/540785363.py:1: RuntimeWarning: invalid value encountered in sqrt
  q = np.sqrt( ( 1 / (4*(H**2)) ) - ( (beta*S) / (U) ) )
